In [ ]:
import sys  
sys.path.append('../models/')
import ExampleModels as em
import torchvision
from torchvision import transforms
import torch

In [ ]:
def get_dataset():
    data_dir = '../dataset'
    train_dataset = torchvision.datasets.MNIST(data_dir, train=True, download=True)
    test_dataset  = torchvision.datasets.MNIST(data_dir, train=False, download=True)
    train_transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    train_dataset.transform = train_transform
    test_dataset.transform = test_transform

    m=len(train_dataset)

    #train_data, val_data = random_split(train_dataset, [int(m-m*0.2), int(m*0.2)])
    batch_size=32

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    #valid_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,shuffle=True)
    
    return train_dataset, test_dataset, train_loader, test_loader

train_dataset, test_dataset, train_loader, test_loader = get_dataset()

In [ ]:
# train distribution scaled model and evaluate
max_embed_size = 40
min_p = 0.1
ae = em.MNISTAutoEncoder(max_embed_size,use_sq_dr= True,dr_min_p= min_p, scale_output=True)

num_epochs = 20
diz_loss = {'train_loss':[],'val_loss':[]}
for epoch in range(num_epochs):
    train_loss =ae.train_epoch(train_loader)
    val_loss = ae.test_epoch(test_loader)
    print('EPOCH {}/{} \t train loss {} \t val loss {}'.format(epoch + 1, num_epochs,train_loss,val_loss))
    diz_loss['train_loss'].append(float(train_loss))
    diz_loss['val_loss'].append(float(val_loss))

In [ ]:
# train a normal model and evaluate

max_embed_size = 40
min_p = 0.1
ae = em.MNISTAutoEncoder(max_embed_size,use_sq_dr= True,dr_min_p= min_p, scale_output=False)

num_epochs = 20
niz_loss = {'train_loss':[],'val_loss':[]}
for epoch in range(num_epochs):
    train_loss =ae.train_epoch(train_loader)
    val_loss = ae.test_epoch(test_loader)
    print('EPOCH {}/{} \t train loss {} \t val loss {}'.format(epoch + 1, num_epochs,train_loss,val_loss))
    niz_loss['train_loss'].append(float(train_loss))
    niz_loss['val_loss'].append(float(val_loss))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(diz_loss['train_loss'],label='scaled')
plt.plot(niz_loss['train_loss'],label='normal')
plt.plot(diz_loss['val_loss'],label='scaled_val')
plt.plot(niz_loss['val_loss'],label='normal_val')
plt.legend()
plt.show()